# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 10:38AM,257336,20,ATR-I000100,"HVL, LLC dba Atrium Innovations",Released
1,Feb 14 2023 10:27AM,257334,10,CLI00468.1,"CLINUVEL, Inc.",Released
2,Feb 14 2023 10:24AM,257333,16,SHL-9157834,"SHL Pharma, LLC",Released
3,Feb 14 2023 10:10AM,257331,10,0086379087,ISDIN Corporation,Released
4,Feb 14 2023 10:10AM,257331,10,0086379090,ISDIN Corporation,Released
5,Feb 14 2023 10:10AM,257331,10,0086379091,ISDIN Corporation,Released
6,Feb 14 2023 10:10AM,257331,10,0086379092,ISDIN Corporation,Released
7,Feb 14 2023 10:10AM,257331,10,0086379093,ISDIN Corporation,Released
8,Feb 14 2023 10:10AM,257331,10,0086379094,ISDIN Corporation,Released
9,Feb 14 2023 10:10AM,257331,10,0086379089,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257331,Released,31
47,257332,Released,1
48,257333,Released,1
49,257334,Released,1
50,257336,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257331,NaN,31.0
257332,NaN,1.0
257333,NaN,1.0
257334,NaN,1.0
257336,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257238,0.0,11.0
257240,1.0,2.0
257242,0.0,3.0
257254,0.0,1.0
257256,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257238,0,11
257240,1,2
257242,0,3
257254,0,1
257256,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257238,0,11
1,257240,1,2
2,257242,0,3
3,257254,0,1
4,257256,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257238,,11
1,257240,1,2
2,257242,,3
3,257254,,1
4,257256,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations"
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc."
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC"
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation
34,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC"
35,Feb 14 2023 10:06AM,257330,21,"NBTY Global, Inc."
36,Feb 14 2023 10:04AM,257329,10,Emerginnova
37,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc."
38,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC"
50,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",,1
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",,1
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",,1
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,,31
4,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC",,1
5,Feb 14 2023 10:06AM,257330,21,"NBTY Global, Inc.",,1
6,Feb 14 2023 10:04AM,257329,10,Emerginnova,,1
7,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",,1
8,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",3,9
9,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",1,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",1,
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",1,
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",1,
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,31,
4,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC",1,
5,Feb 14 2023 10:06AM,257330,21,"NBTY Global, Inc.",1,
6,Feb 14 2023 10:04AM,257329,10,Emerginnova,1,
7,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",1,
8,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",9,3
9,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",4,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",1,
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",1,
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",1,
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,31,
4,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",1.0,NaN
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",1.0,NaN
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,31.0,NaN
4,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 10:38AM,257336,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
1,Feb 14 2023 10:27AM,257334,10,"CLINUVEL, Inc.",1.0,0.0
2,Feb 14 2023 10:24AM,257333,16,"SHL Pharma, LLC",1.0,0.0
3,Feb 14 2023 10:10AM,257331,10,ISDIN Corporation,31.0,0.0
4,Feb 14 2023 10:08AM,257332,10,"Snap Medical Industries, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6175241,227.0,3.0
12,771957,0.0,3.0
15,1029092,40.0,24.0
16,257333,1.0,0.0
19,2315563,75.0,12.0
20,514628,1.0,1.0
21,514596,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6175241,227.0,3.0
1,12,771957,0.0,3.0
2,15,1029092,40.0,24.0
3,16,257333,1.0,0.0
4,19,2315563,75.0,12.0
5,20,514628,1.0,1.0
6,21,514596,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,227.0,3.0
1,12,0.0,3.0
2,15,40.0,24.0
3,16,1.0,0.0
4,19,75.0,12.0
5,20,1.0,1.0
6,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,227.0
1,12,Released,0.0
2,15,Released,40.0
3,16,Released,1.0
4,19,Released,75.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,3.0,3.0,24.0,0.0,12.0,1.0,0.0
Released,227.0,0.0,40.0,1.0,75.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,3.0,3.0,24.0,0.0,12.0,1.0,0.0
1,Released,227.0,0.0,40.0,1.0,75.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,3.0,3.0,24.0,0.0,12.0,1.0,0.0
1,Released,227.0,0.0,40.0,1.0,75.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()